# SIECGALE - accès à la BDD

Ce notebook permet d'insérer des articles dans la base de données SQLite, elle-même étant utilisée par le front en streamlit.

**Sources :**
- https://share.streamlit.io/ines/spacy-streamlit-demo/app.py
- https://spacy.io/models/fr
- https://sqlite-utils.datasette.io/en/stable/python-api.html#getting-started
- https://spacy.io/usage/visualizers

**Reste à faire**
- sexifier l'interface
- optimiser (cache) les fonctions qui n'ont pas besoin de s'actualiser à chaque "rerun"
- ajouter une fonction de recherche par SIREN ou nom d'entreprise pour en montrer tous les articles

In [1]:
import datetime

import pandas as pd
import numpy as np
from sqlite_utils import Database

from script import génère_faux_articles

In [6]:
db = Database("base_sinistre.db", recreate=True)
schéma = {'date_article': datetime.date, 'siren': str, 'entreprise': str, 'statut_entreprise': str, 'contenu': str}
db.create_table('sinistres', columns=schéma, if_not_exists=True)

<Table sinistres (date_article, siren, entreprise, statut_entreprise, contenu)>

In [7]:
articles = génère_faux_articles(30)
pd.DataFrame(articles).tail(3)

,date_article,entreprise,siren,statut_entreprise,contenu
27,2022-01-08,Valette,301 517 747,Client,Le feu s'est déclaré dans la nuit du samedi 08...
28,2022-03-13,Duhamel,175 518 677,Entreprise sans interaction avec Generali,Le feu s'est déclaré dans la nuit du dimanche ...
29,2022-03-15,Leroux,925 975 624,Ancien client,Le feu s'est déclaré dans la nuit du mardi 15 ...


In [8]:
db['sinistres'].insert_all(articles)

<Table sinistres (date_article, siren, entreprise, statut_entreprise, contenu)>

Requête pour les afficher :

In [9]:
début, fin = datetime.date(2022, 4, 1), datetime.date.today()

requête = f"""
SELECT *
FROM sinistres
WHERE date_article > '{début}' AND date_article < '{fin}'
"""

df = pd.DataFrame(db.query(requête))
df

,date_article,siren,entreprise,statut_entreprise,contenu
0,2022-04-06,166 264 106,Rocher S.A.R.L.,Client,Le feu s'est déclaré dans la nuit du mercredi ...
